## Part 1 - Data Preprocessing

In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13]
y = dataset.iloc[:, 13]

In [3]:
#Create dummy variables
geography=pd.get_dummies(X["Geography"],drop_first=True)
gender=pd.get_dummies(X['Gender'],drop_first=True)

In [4]:
## Concatenate the Data Frames

X=pd.concat([X,geography,gender],axis=1)


In [5]:
## Drop Unnecessary columns
X=X.drop(['Geography','Gender'],axis=1)

In [6]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)


In [7]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Perform Hyperparameter Optimization

In [8]:
from tensorflow import keras
from tensorflow.keras import layers
from keras_tuner.tuners import RandomSearch

In [9]:
def built_model(hp):
    model= keras.Sequential()
    for i in range(hp.Int('number of layers',2,20)): #for the number of hidden layers 
        model.add(layers.Dense(
            units=hp.Int("units_" + str(i), min_value=32, max_value=512, step=32), #for no of neurons in hidden layers
                activation="relu"))
        model.add(layers.Dense(units=1,activation='linear')) # this is output layers and no of nuerons =1
        model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])), # for choosing learning rate
            loss='binary_crossentropy',     # for linear we choose loss function-- mean_squared_error, classification-binary_crossentropy
            metrics=['accuracy'])  # for linear matrics----> mean_absolute error, claasification matics---> [accuracy]
        return model    
        

In [10]:
tuner= RandomSearch(built_model,
                   objective='val_accuracy',  #classification----> val_accuracy
                   max_trials=5,
                   executions_per_trial=3,
                   directory='project',
                   project_name='churn') 

INFO:tensorflow:Reloading Oracle from existing project project\churn\oracle.json
INFO:tensorflow:Reloading Tuner from project\churn\tuner0.json


In [11]:
#You can print a summary of the search space:
tuner.search_space_summary()

Search space summary
Default search space size: 3
number of layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [12]:
#Then, start the search for the best hyperparameter configuration. The call to search has the same signature as model.fit().
tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test))

INFO:tensorflow:Oracle triggered exit


In [13]:
#When search is over, you can retrieve the best model(s):
models_tuner = tuner.get_best_models(num_models=2)


In [14]:
#Or print a summary of the results:
tuner.results_summary() #result of best model

Results summary
Results in project\churn
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
number of layers: 7
units_0: 512
learning_rate: 0.001
Score: 0.8425000111262003
Trial summary
Hyperparameters:
number of layers: 8
units_0: 480
learning_rate: 0.001
Score: 0.8215000033378601
Trial summary
Hyperparameters:
number of layers: 11
units_0: 480
learning_rate: 0.01
Score: 0.8008333444595337
Trial summary
Hyperparameters:
number of layers: 5
units_0: 416
learning_rate: 0.01
Score: 0.7986666758855184
Trial summary
Hyperparameters:
number of layers: 5
units_0: 192
learning_rate: 0.0001
Score: 0.7883333365122477


## Perform Hyperparameter Optimization: M-2

In [15]:

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, LeakyReLU, BatchNormalization, Dropout
from keras.activations import relu, sigmoid

In [16]:
def create_model(layers, activation):
    model = Sequential()
    for i, nodes in enumerate(layers):
        if i==0:
            model.add(Dense(nodes,input_dim=X_train.shape[1])) #at first hidden layer it will take input_dim= X_train.shape[1]=11
            model.add(Activation(activation)) # add activation 
            model.add(Dropout(0.3)) # here we are using dropout to prevent overfitting
        else:
            model.add(Dense(nodes))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
            
    model.add(Dense(units = 1, kernel_initializer= 'glorot_uniform', activation = 'sigmoid')) # Note: no activation beyond this point
    
    model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy']) # since this is binary classification problem
    return model


In [17]:
#creating classifer model
model = KerasClassifier(build_fn=create_model, verbose=0) 

In [18]:
layers = [[20], [40, 20], [45, 30, 15]]  #here we are proving layers in first-- one hidden layer with 20 neurons, 2nd-->2 hidden layr with 20 and 40 neurons 
activations = ['sigmoid', 'relu']  #here we are provinding two activation functions for hidden layers 
param_grid = dict(layers=layers, activation=activations, batch_size = [128, 256], epochs=[30]) #pass alway in form of dictionary
grid = GridSearchCV(estimator=model, param_grid=param_grid,cv=5) 


In [19]:
# We will fit the model
grid_result = grid.fit(X_train, y_train)

In [20]:
print(grid_result.best_score_, grid_result.best_params_)

0.8548749923706055 {'activation': 'relu', 'batch_size': 128, 'epochs': 30, 'layers': [45, 30, 15]}


In [21]:
y_pred=grid.predict(X_test)


In [22]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
accuracy=accuracy_score(y_test,y_pred)
print('The accuracy of the model is', accuracy)
cm=confusion_matrix(y_test,y_pred)
print(cm)
print(classification_report(y_test,y_pred))

The accuracy of the model is 0.862
[[1529   66]
 [ 210  195]]
              precision    recall  f1-score   support

           0       0.88      0.96      0.92      1595
           1       0.75      0.48      0.59       405

    accuracy                           0.86      2000
   macro avg       0.81      0.72      0.75      2000
weighted avg       0.85      0.86      0.85      2000

